### Comparação entre os modelos de classificação

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report)

# --- 1. Carregamento e Alinhamento Inicial dos Dados ---
try:
    df_full = pd.read_csv('df_com_outliers.csv')
    df_clean = pd.read_csv('df_sem_outliers.csv')
except FileNotFoundError:
    print("Arquivos 'df_com_outliers.csv' e/ou 'df_sem_outliers.csv' não encontrados.")
    exit()

# --- 2. Encontrar Intersecção e Alinhar DataFrames ---
# Encontrar os barcodes comuns a ambos os dataframes
common_barcodes = np.intersect1d(df_full['barcode_hashed'], df_clean['barcode_hashed'])

# Filtrar ambos os dataframes para conter apenas os barcodes comuns
df_full_aligned = df_full[df_full['barcode_hashed'].isin(common_barcodes)].copy()
df_clean_aligned = df_clean[df_clean['barcode_hashed'].isin(common_barcodes)].copy()

# --- 3. Divisão Única dos Dados (baseada nos dados alinhados) ---
# Usaremos o df_clean_aligned para a divisão, pois representa o conjunto de dados válidos após todo o tratamento
X = df_clean_aligned.drop('has_failure', axis=1)
y = df_clean_aligned['has_failure']

# A divisão agora é feita em um X que AINDA CONTÉM o barcode_hashed
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# --- 4. Preparar os Conjuntos de Dados para Cada Grupo usando o barcode_hashed ---

# Grupo 1: Modelos de Árvore (usam dados com outliers)
# Usamos o merge para garantir o alinhamento perfeito pelas chaves de treino e teste
X_train_tree = pd.merge(X_train[['barcode_hashed']], df_full_aligned, on='barcode_hashed', how='inner').drop('has_failure', axis=1)
X_test_tree = pd.merge(X_test[['barcode_hashed']], df_full_aligned, on='barcode_hashed', how='inner').drop('has_failure', axis=1)

# Grupo 2: Modelos Lineares/SVM (usam os dados já divididos do conjunto limpo)
X_train_linear = X_train
X_test_linear = X_test

# --- 5. Remover a Coluna de ID e Aplicar Standardização ---

# Agora removemos o barcode_hashed dos 4 dataframes de features
id_col = 'barcode_hashed'
X_train_tree_final = X_train_tree.drop(id_col, axis=1)
X_test_tree_final = X_test_tree.drop(id_col, axis=1)
X_train_linear_final = X_train_linear.drop(id_col, axis=1)
X_test_linear_final = X_test_linear.drop(id_col, axis=1)

# Standardização (apenas para o grupo 2)
scaler = StandardScaler()
X_train_linear_scaled = scaler.fit_transform(X_train_linear_final)
X_test_linear_scaled = scaler.transform(X_test_linear_final)

# --- 6. Treinamento e Avaliação Unificados ---
models = {
    'Random Forest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(random_state=42, probability=True)
}

results_list = []

for name, model in models.items():
    print(f"\n----- Avaliando: {name} -----")
    
    # y_train e y_test são os mesmos para todos
    if name in ['Random Forest', 'XGBoost']:
        model.fit(X_train_tree_final, y_train)
        y_pred = model.predict(X_test_tree_final)
        y_pred_proba = model.predict_proba(X_test_tree_final)[:, 1]
    else:
        model.fit(X_train_linear_scaled, y_train)
        y_pred = model.predict(X_test_linear_scaled)
        y_pred_proba = model.predict_proba(X_test_linear_scaled)[:, 1]

    # Calcular e armazenar métricas
    results_list.append({
        'Modelo': name,
        'Acurácia': accuracy_score(y_test, y_pred),
        'Precisão': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_pred_proba)
    })
    
    print(classification_report(y_test, y_pred))
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("-" * 30)

# --- 7. Comparação Final ---
results_df = pd.DataFrame(results_list)
print("\n===== Tabela Comparativa de Desempenho dos Modelos =====")
print(results_df.sort_values(by='F1-Score', ascending=False).to_string())


----- Avaliando: Random Forest -----
              precision    recall  f1-score   support

           0       0.65      0.53      0.59      2406
           1       0.74      0.82      0.78      3901

    accuracy                           0.71      6307
   macro avg       0.69      0.68      0.68      6307
weighted avg       0.71      0.71      0.71      6307

Matriz de Confusão:
[[1285 1121]
 [ 698 3203]]
------------------------------

----- Avaliando: XGBoost -----


C:\Users\Denil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [12:05:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.62      0.57      0.60      2406
           1       0.75      0.79      0.77      3901

    accuracy                           0.70      6307
   macro avg       0.69      0.68      0.68      6307
weighted avg       0.70      0.70      0.70      6307

Matriz de Confusão:
[[1376 1030]
 [ 831 3070]]
------------------------------

----- Avaliando: Logistic Regression -----
              precision    recall  f1-score   support

           0       0.55      0.19      0.29      2406
           1       0.65      0.90      0.75      3901

    accuracy                           0.63      6307
   macro avg       0.60      0.55      0.52      6307
weighted avg       0.61      0.63      0.58      6307

Matriz de Confusão:
[[ 467 1939]
 [ 376 3525]]
------------------------------

----- Avaliando: SVM -----
              precision    recall  f1-score   support

           0       0.60      0.22      0.32      2406
         